# Introduction

- Most APIs require authentication
- APIs also use authentication to perform __rate limiting__
    - to ensure it remains avilable and responsive, APIs will prevent you from making too many requests in too short a time, i.e. restriction rate limiting
- In this mission we're going to take a look at the [GitHub API](https://developer.github.com/v3/) 

# API Authentication

- To authenticate with the GitHub API, we'll need to us an access token
    - a credential generated via GitHub
    - is a string that the API can read and associate with your account
- Using a token is preferable to a username and password
    - typically, you'll be accessing an API from a script, if username/passsword is leaked someone else can take over your account
    - Access tokens can have scopes and specific permissions
        - Using read-access-only tokens in potentially insecure or shared scripts gives you more control over security
- Will need to pass your token to the GitHub API via an _Authorization header_
    - headers contain metadata about the request
    - can use Python's `requests` libraru to make a dictionary of headers
- Should __never__ share your token with anyone you don't want to have access to your account

In [2]:
import requests

# Create a dictionary of headers containing our Authorization header.
headers = {"Authorization": "INSERT TOKEN HERE"}

# Make a GET request to the GitHub API with our headers.
# This API endpoint will give us details about Vik Paruchuri.
response = requests.get("https://api.github.com/users/VikParuchuri", headers=headers)

# Print the content of the response.  As you can see, this token corresponds to the account of Vik Paruchuri.
print(response.json())

{'message': 'Bad credentials', 'documentation_url': 'https://developer.github.com/v3'}


In [ ]:
# Create a dictionary of headers containing our Authorization header.
headers = {"Authorization": "INSERT TOKEN HERE"}

# Make an authenticated request to https://api.github.com/users/VikParuchuri/orgs. This will give us a list of the organizations a GitHub user belongs to
response = requests.get("https://api.github.com/users/VikParuchuri/orgs", headers=headers)

# Assign the JSON content of the response to orgs (you can get this with response.json())
orgs = response.json()

print(orgs)

# Endpoints and Objects

- APIs usually let us retrive information about specific objects in a database
- Based on the example above, we could also retrive information about other GitHub users through the same endpoint

In [ ]:
# We've loaded headers in.
# Use the endpoint https://api.github.com/users/torvalds with the same headers from before to get information about Linus Torvalds.
response = requests.get('https://api.github.com/users/torvalds', headers=headers)

torvalds = response.json()

print(torvalds)

# Other Objects

In [ ]:
# Enter your answer here.
# Make a GET request to the https://api.github.com/repos/octocat/Hello-World endpoint
response = requests.get('https://api.github.com/repos/octocat/Hello-World', headers=headers)

hello_world = response.json()

print(hello_world)

# Pagination

- Sometimes a request can return a lot of objects
    - example: listing out all of a user's repositories
- __Pagination__: means that the API provider will only return a certain number of records per page
    - can specify the page number you want to access
    - to access all the pages, you'll need to write a loop
- To get repositories a user has _starred_ we can use the following API endpoint:
`https://api.github.com/users/VikParuchuri/starred`
    - can add two pagination query paramters - `page` and `per_page`
        - `page` is the page we want to access
        - `per_page` is the number of records we want to see on each page
            - typically a cap on this
            
         

In [ ]:
# Get the second page of repositories that Vik Paruchuri 
# starred from the https://api.github.com/users/VikParuchuri/starred endpoint.
params = {"per_page": 50, "page": 2}
response = requests.get("https://api.github.com/users/VikParuchuri/starred", headers=headers, params=params)
page2_repos = response.json()

# User-Level Endpoints

In [ ]:
# make a GET request to user to get info on the user the authentication token is for
response = requests.get("https://api.github.com/user", headers=headers)
user = response.json()

# POST requests

- few other types of API requests
- `POSTS` requests to send information and to create objects on API's server
    - For GitHub, can use `POST` to create new repos
- Not all endpoints accept a `POST` request, and not all will accept a `GET` request
- Can make `POST` requests using `requests.post`

### EXAMPLE

`payload = {"name": "test"}
 requests.post("https://api.github.com/user/repos", json=payload)`
 
 - code above will create a new repo named `test` under the account of the authenticated user
 - [Here is GitHub's API documentation for repos](https://developer.github.com/v3/repos/)
 - A successful `POST` request will usually return `201` status code
     - indicates that it was able to create the object on the server
  

In [ ]:
# Create the data we'll pass into the API endpoint.  While this endpoint only requires the "name" key, there are other optional keys.
payload = {"name": "learning-about-apis"}

# We need to pass in our authentication headers!
response = requests.post("https://api.github.com/user/repos", json=payload, headers=headers)

status = response.status_code

# PUT/PATCH Requests

- `PATCH`/`PUT` requests come into play when we want to update an existing object
- `PATCH` = when we want to change a few attributes of an object but don't want to resend the entire object to the server
    - example: change the name of the repo
- `PUT` = send the complete object we're revising as a replacement for the server's existing version
- In practive, API developers don't always respect this convention
    - sometimes API endpoints that accept `PUT` requests will treat them like `PATCH` requests and not require us to send the whole object back

`payload = {"description": "The best repository ever!", "name": "test"}
response = requests.patch("https://api.github.com/repos/VikParuchuri/test", json=payload)`

Code above will change the description of the test repo to The best repository ever!

- A successful `PATCH` request will usually return a `200` status code

In [ ]:
# Make a PATCH request to the https://api.github.com/repos/VikParuchuri/learning-about-apis 
# endpoint that changes the description to Learning about requests!.
payload = {"description": "Learning about requests!", "name": "learning-about-apis"}
response = requests.patch("https://api.github.com/repos/VikParuchuri/learning-about-apis", json=payload, headers=headers)

status = response.status_code

# DELETE Requests

- `DELETE` request removes objects from the server
    - example: can be used to remove repos
    
`response = requests.delete("https://api.github.com/repos/VikParuchuri/test")`

The code above will delete the `test` repository from GitHub
- a successful `DELETE` will usually return a `204` status code indicating that it successfully deleted the object
- use them very carefully!

In [ ]:
# Make a DELETE request to the https://api.github.com/repos/VikParuchuri/learning-about-apis 
# endpoint
response = requests.delete('https://api.github.com/repos/VikParuchuri/learning-about-apis', headers=headers)

status = response.status_code